<a href="https://colab.research.google.com/github/marloquemegusta/filtros_para_imagenes/blob/master/pruebas_filtros_imagenes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/marloquemegusta/filtros_para_imagenes.git

In [ ]:
import cv2
from PIL import Image
from google.colab.patches import cv2_imshow
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from PIL import Image, ImageCms
from skimage import feature
from sklearn.cluster import MiniBatchKMeans
from skimage.util import random_noise
import glob
import imutils
from google.colab.patches import cv2_imshow
import matplotlib

In [ ]:
image_paths=glob.glob("/content/filtros_para_imagenes/imagenes/*")
images=[]
widths=[]
heights=[]
for image_path in image_paths:
  image=cv2.imread(image_path)
  image=imutils.resize(image,1000)
  images.append(image)
max_height=max([image.shape[0] for image in images])
for i,image in enumerate(images):
  pad_amount=int((max_height-image.shape[0])/2)
  images[i]=np.pad(image,((pad_amount,pad_amount),(0,0),(0,0)))
  images[i]=cv2.resize(images[i],(1000,max_height))
  
images=np.array(images)
W=images.shape[2]
H=images.shape[1]

In [ ]:
stack=np.zeros((max_height,1,3))
for image in  images:
  stack=np.hstack((stack,image))
cv2_imshow(stack)

# Expansión del rango dinámico

En esta sección exploraremos varias maneras de expandir el rango dinámico de las imágenes para obtener un resultado más llamativo. Algo similar a lo que hace el algoritmo HDR. 

Las funciones descritas a continuación transforman la imagen de RGB a otro espacio de color para que la ecualización de histograma tenga sentido. Si aplicásemos la ecualización de histograma a alguno o a los tres canales en RGB obtendríamos una distorsión en los colores, no expansión en el rango dinámico. A continuación se enseña un ejemplo de qué pasaría al ecualizar los canales de color en RGB

In [ ]:
stack=np.zeros((max_height*2,1,3))
for i,image in enumerate(images):
  image_eq_hist=image.copy()
  for channel in range(3):
    image_eq_hist[:,:,channel]=cv2.equalizeHist(image[:,:,channel])
  stack=np.hstack((stack,np.vstack((image,image_eq_hist))))
cv2_imshow(stack)

In [ ]:
# en esta función convertimos la imagen a HSV y, en este formato, ecualizamos el
# canal V (value). Este canal es el que representa la luminosidad, siendo 0 
# un pixel totalmente negro y 255 un pixel totalmente blanco
def valueHistEq(img):
  img=img.copy() 
  HSV=cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
  HSV[:, :, 2] = cv2.equalizeHist(HSV[:, :, 2])
  BGR=cv2.cvtColor(HSV,cv2.COLOR_HSV2BGR,img)
  return BGR
# en este caso hacemos lo mismo que en el anterior, pero esta vez no ecualizamos
# el histograma de la imagen entera, sino que lo hacemos por bloques aplicando 
# un umbral máximo que no se puede superar (clipLimit)
def clipValueHistEq(img,clipLimit=2.0,girdSize=(8,8)):
  img=img.copy()
  HSV=cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
  HSV[:, :, 2] = cv2.equalizeHist(HSV[:, :, 2])
  BGR=cv2.cvtColor(HSV,cv2.COLOR_HSV2BGR,img)
  return BGR

# Convertimos la imagen al formato YCrCb y ecualizamos el canal Y, también
# correspondiente al brillo o luminosidad
def yHistEq(img):
  img=img.copy()
  ycrcb=cv2.cvtColor(img,cv2.COLOR_BGR2YCR_CB)
  ycrcb[:, :, 0] = cv2.equalizeHist(ycrcb[:, :, 0])
  BGR=cv2.cvtColor(ycrcb,cv2.COLOR_YCR_CB2BGR,img)
  return BGR


def clipYHistEq(img,clipLimit=2.0,girdSize=(8,8)):
  img=img.copy()
  ycrcb=cv2.cvtColor(img,cv2.COLOR_BGR2YCR_CB)
  clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
  ycrcb[:, :, 0] = clahe.apply(ycrcb[:, :, 0].astype("uint8"))
  BGR=cv2.cvtColor(ycrcb,cv2.COLOR_YCR_CB2BGR,img)
  return BGR

In [ ]:
#ecualización del histograma del canal V
stack=np.zeros((max_height*2,1,3))
for image in images:
  image_eq_hist=valueHistEq(image)
  stack=np.hstack((stack,np.vstack((image,image_eq_hist))))
cv2_imshow(stack)

In [ ]:
#ecualización del histograma del canal Y
stack=np.zeros((max_height*2,1,3))
for image in images:
  image_eq_hist=yHistEq(image)
  stack=np.hstack((stack,np.vstack((image,image_eq_hist))))
cv2_imshow(stack)

In [ ]:
#ecualización del histograma del canal V por bloques
stack=np.zeros((max_height*2,1,3))
for image in images:
  image_eq_hist=clipValueHistEq(image)
  stack=np.hstack((stack,np.vstack((image,image_eq_hist))))
cv2_imshow(stack)

In [ ]:
#ecualización del histograma del canal Y por bloques
stack=np.zeros((max_height*2,1,3))
for image in images:
  image_eq_hist=clipYHistEq(image)
  stack=np.hstack((stack,np.vstack((image,image_eq_hist))))
cv2_imshow(stack)

# Detección de bordes

In [ ]:
def get_colorized_edges(img,sigma=2):
  img_gray= cv2.cvtColor(np.array(img), cv2.COLOR_BGR2GRAY)
  edges=feature.canny(img_gray,sigma=sigma).astype("uint8")
  kernel=np.ones((3,3))
  edges=cv2.dilate(edges,kernel,iterations=1)
  edged_im1=cv2.bitwise_and(np.asarray(img),np.asarray(img),mask=edges)
  return edged_im1.astype("uint8")

def draw_edges_over_image(img,sigma=2,color=(0,0,0),offset=0,thikness=0):
  img_gray= cv2.cvtColor(np.array(img), cv2.COLOR_BGR2GRAY)
  edges=feature.canny(img_gray,sigma=sigma).astype("uint8")
  kernel=np.ones((3,3))
  edges=cv2.dilate(edges,kernel,iterations=thikness)
  edges=np.roll(edges,offset,axis=1)
  edges_rgb=cv2.bitwise_and(np.ones(np.array(img).shape)*127,
                            np.ones(np.array(img).shape)*127,
                            mask=edges).astype("uint8")
  edges_rgb=np.where(edges_rgb==(127,127,127),color,(127,127,127))
  edged_im2=np.where(edges_rgb!=(127,127,127),edges_rgb,np.array(img))
  return edged_im2.astype("uint8")

Aquí podemos escoger como parámetro sigma, que representa la std del filtro gausiano que se aplica a la imagen antes de computar
los bordes. Cuanto mayor sigma, menos bordes saldrán 

In [ ]:
#obteniendo bordes coloreados
stack=np.zeros((max_height*3,1,3))
for image in images:
  image_colorized_borders_1=get_colorized_edges(image)
  image_colorized_borders_2=get_colorized_edges(image,3)
  stack=np.hstack((stack,np.vstack((image,image_colorized_borders_1,image_colorized_borders_2))))
cv2_imshow(stack)

En este caso tenemos el parámetro sigma, el color del cual queremos pintar los bordes, el desplazamiento, en número de píxeles a la derecha, que queremos hacer de los bordes y su grosor

In [ ]:
# obteniendo bordes de la imagen
stack=np.zeros((max_height*3,1,3))
for image in images:
  image_borders_1=draw_edges_over_image(image,
                                        color=(255,255,255),
                                        offset=0,
                                        thikness=1,
                                        sigma=3)
  image_borders_2=draw_edges_over_image(image,
                                        color=(255,255,255),
                                        offset=50,
                                        thikness=1,
                                        sigma=3)
  stack=np.hstack((stack,np.vstack((image,image_borders_1,image_borders_2))))
cv2_imshow(stack)

# Reducción del espacio de color

In [ ]:
def reduce_color_space(img,ncolors=10):
  imlab = cv2.cvtColor(np.array(img), cv2.COLOR_BGR2LAB)
  (h, w) = imlab.shape[:2]
  imlab = imlab.reshape((imlab.shape[0] * imlab.shape[1], 3))
  kmeans = MiniBatchKMeans(n_clusters =ncolors)
  labels = kmeans.fit_predict(imlab)
  quant = kmeans.cluster_centers_.astype("uint8")[labels]
  quant = quant.reshape((h, w, 3))
  return cv2.cvtColor(quant, cv2.COLOR_LAB2BGR)

En este caso lo que hacemos es aplicar una agrupación de los colores usando Kmeans. El número de clusters determina el número de colores en la imagen final. Una vez el algoritmo de Kmeans agrupa los colores, coloreamos la imagen utilizando, para cada color, el color más cercano de entre los centroides del Kmeans.
Es importante aclarar que, dado que Kmeans busca grupos con la mínima distancia entre sus integrantes, hemos de transformar la imagen a un espacio de color donde haya una métrica de distancia relevante. Para esto utilizamos el espacio de color [LAB](https://es.wikipedia.org/wiki/Espacio_de_color_Lab)

In [ ]:
# reduciendo el espacio de color
stack=np.zeros((max_height*4,1,3))
for image in images:
  image_3_colors=reduce_color_space(image,3)
  image_8_colors=reduce_color_space(image,8)
  image_64_colors= reduce_color_space(image,64)
  stack=np.hstack((stack,np.vstack((image,image_3_colors,image_8_colors,image_64_colors))))
cv2_imshow(stack)

# otros filtros

In [ ]:
def shift_rgb(img,offset=10):
  img=img.copy()
  for channel in range(img.shape[2]):
    img[:,:,channel]=np.roll(img[:,:,channel],offset*channel,axis=1)
  return img

def pixel_art(img,width=64, n_colors=32, border_thikness=0, sigma=1, orig_size=True):
  img=img.copy()
  img=reduce_color_space(img,n_colors)
  img_mini=imutils.resize(img,width=width)
  img_mini_edges=draw_edges_over_image(img_mini,thikness=border_thikness,sigma=sigma)
  if orig_size:
    img_orig_size_edges= cv2.resize(img_mini_edges,(img.shape[1],img.shape[0]),
                                  interpolation = cv2.INTER_NEAREST)
    return img_orig_size_edges
  return img_mini_edges

En esta sección compondremos varios efectos más, ya sea efectos que no encajan en las categorías anteriores o algunos que se forman por la adición de varios de los anteriores.

En el primero trataremos de crear imágenes con estilo "pixel art", es decir, baja resolución, espacio de color reducido y bordes muy destacados. Los parámetros que podemos modificar son el tamaño de la imagen en píxeles (de ancho), el número de colores, el grosor de los bordes y la sigma del filtro gausiano.

In [ ]:
# reduciendo el espacio de color
stack=np.zeros((max_height*3,1,3))
for image in images:
  pixel_art_64=pixel_art(image,
                         sigma=1,
                         width=64,
                         n_colors=16)
  pixel_art_128=pixel_art(image,
                         sigma=2,
                         width=128,
                         n_colors=16)
  stack=np.hstack((stack,np.vstack((image,pixel_art_64,pixel_art_128))))
cv2_imshow(stack)

En este siguiente filtro simplemente se desplazan los canales rojo, verde y azul de una imagen RGB para crear este efecto tan chulo. El único parámetro a ajustar es el desplazamiento que queremos entre los canales

In [ ]:
# desplazando los canales RGB
stack=np.zeros((max_height*3,1,3))
for image in images:
  shifted_channels_10=shift_rgb(image,10)
  shifted_channels_30=shift_rgb(image,30)
  stack=np.hstack((stack,np.vstack((image,shifted_channels_10,shifted_channels_30))))
cv2_imshow(stack)